In [ ]:
!pip install groq
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

In [ ]:
import time
from datasets import load_dataset
import os
from groq import Groq

In [ ]:
ds = load_dataset("ChilleD/SVAMP")
client = Groq(api_key='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')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/675 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/111k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/54.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/700 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

Zero_Shot

In [ ]:
student_system = {"role": "system",
                  "content": "Never forget you are a brilliant intelligent STUDENT who solves complex math problems with help of TEACHER guidance. \
                  YOUR TASK: Together with your TEACHER's help find correct answer to math questions. The QUESTION is VERIFIED and is ALWAYS CORRECT. Never forget your TASK. \
                  You as a STUDENT will get a math question from USER and in return you will write, detailed explaination to answer that question. \
                  Your TEACHER will provide you with feedback based upon what you did. \
                  If it is incorrect at some part you will be told what you did wrong and so you will modify your answer based upon the TEACHER's feedback. \
                  If it is correct, you will not modify your answer and return the same answer just by checking your calculations. \
                  Always follow the given pattern TEMPLATE to answer. \
                  TEMPLATE: \
                  Explanation Body Paragraph\
                  **Final Answer: <SINGLE NUMERICAL ANSWER>**"}

teacher_system = {"role": "system",
                  "content": "Never forget you are an intelligent helpful TEACHER who helps his STUDENT to solve complex math problems. \
                  You will have a question and the answer that student writes. The given QUESTION is VERIFIED and is ALWAYS CORRECT. STICK TO THE GIVEN QUESTION \
                  YOUR TASK: Guide and help your STUDENT to find correct answer and explaination to math questions. Never forget your TASK. \
                  You as a TEACHER will check if the explaination of the answer provided by the STUDENT is correct or not and provide him feedback. \
                  If the explaination is incorrect you will tell which part of the answer did the STUDENT went wrong, whether it is calculation or the approach the student used to get the answer \
                  If the explaination is correct you will praise the STUDENT and tell him to write the answer again. \
                  Also instruct the student every time to follow the given template \
                  TEMPLATE: \
                  Explanation Body Paragraph\
                  **Final Answer: <SINGLE NUMERICAL ANSWER>**"}


In [ ]:
model_name = 'gemma2-9b-it'
def multi_agent_chat(query):
    student_chat = []
    teacher_chat = []
    student_chat.append(student_system)
    teacher_chat.append(teacher_system)
    for i in range(2):
        # print('___________________________________________')
        student_chat.append({"role": "user", "content": query})
        teacher_chat.append({"role": "user", "content": query})
        student_response = client.chat.completions.create(model=model_name, messages=student_chat, temperature=0)
        # print(student_response.choices[0].message.content)
        teacher_chat.append({"role": "user", "content": student_response.choices[0].message.content})
        teacher_response = client.chat.completions.create(model=model_name, messages=teacher_chat, temperature=0, max_tokens=2048)
        # print(teacher_response.choices[0].message.content)
        student_chat.append({"role": "user", "content": teacher_response.choices[0].message.content})
        # student_response = client.chat.completions.create(model=model_name, messages=student_chat, temperature=0, max_tokens=2048)
        # print('___________________________________________')
    # print("FINAL RESPONSE")
    # print(student_response.choices[0].message.content)
    return student_response.choices[0].message.content

In [ ]:
import re
import time
global_counter = 0

def ans_extractor(output):
    global global_counter
    matcha = re.search(r'Answer:\s*([0-9]+)', output)
    if matcha:
        final_answer = matcha.group(1)
        return final_answer
    else:
        global_counter += 1
        return 0

In [ ]:
pt = []
gt = []
counter=0
for i in ds['test']:
    time.sleep(2)
    counter+=1
    print(counter, global_counter)
    query = i['question_concat']
    explaination = multi_agent_chat(query)
    answer = ans_extractor(explaination)
    if answer!=None:
        pt.append(answer)
        gt.append(i['Answer'])

1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 1
21 1
22 1
23 1
24 1
25 1
26 2
27 2
28 2
29 2
30 2
31 2
32 2
33 2
34 2
35 2
36 2
37 2
38 2
39 2
40 2
41 2
42 2
43 2
44 2
45 2
46 2
47 2
48 2
49 2
50 2
51 2
52 2
53 2
54 2
55 2
56 2
57 2
58 2
59 2
60 2
61 2
62 2
63 2
64 2
65 2
66 2
67 2
68 2
69 2
70 2
71 2
72 2
73 2
74 2
75 2
76 3
77 3
78 3
79 3
80 3
81 3
82 3
83 3
84 3
85 3
86 3
87 3
88 3
89 3
90 3
91 4
92 4
93 4
94 4
95 4
96 4
97 4
98 4
99 4
100 4
101 4
102 4
103 4
104 4
105 4
106 4
107 4
108 4
109 5
110 5
111 5
112 5
113 5
114 5
115 6
116 6
117 6
118 6
119 6
120 6
121 6
122 6
123 6
124 6
125 6
126 6
127 6
128 6
129 6
130 6
131 6
132 6
133 6
134 6
135 6
136 6
137 6
138 6
139 6
140 7
141 7
142 7
143 7
144 7
145 7
146 7
147 7
148 7
149 7
150 7
151 7
152 7
153 7
154 7
155 7
156 7
157 7
158 7
159 7
160 7
161 7
162 7
163 7
164 7
165 7
166 7
167 7
168 8
169 8
170 8


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `gemma2-9b-it` in organization `org_01j89nvkdne3kvm03vm5j8m2j7` on : Limit 500000, Used 500002, Requested 547. Please try again in 1m34.872999999s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [ ]:
from sklearn.metrics import accuracy_score
print(global_counter)
accuracy = accuracy_score(gt, pt)
print(f'Accuracy: {accuracy:.2f}')

8
Accuracy: 0.89


In [ ]:
student_system = {"role": "system",
                  "content": "Never forget you are a brilliant intelligent STUDENT who solves complex math problems with help of TEACHER guidance. \
                  YOUR TASK: Together with your TEACHER's help find correct answer to math questions. The QUESTION is VERIFIED and is ALWAYS CORRECT. Never forget your TASK. \
                  You as a STUDENT will get a math question from USER and in return you will write, detailed explaination to answer that question. \
                  Your TEACHER will provide you with feedback based upon what you did. \
                  If it is incorrect at some part you will be told what you did wrong and so you will modify your answer based upon the TEACHER's feedback. \
                  If it is correct, you will not modify your answer and return the same answer just by checking your calculations. \
                  Always follow the given pattern TEMPLATE to answer. \
                  TEMPLATE: \
                  Explanation Body Paragraph\
                  **Final Answer: <SINGLE NUMERICAL ANSWER>** \
                  Make sure the last line of any response is ALWAYS - **Final Answer: <SINGLE NUMERICAL ANSWER>** "}

teacher_system = {"role": "system",
                  "content": "Never forget you are an intelligent helpful TEACHER who helps his STUDENT to solve complex math problems. \
                  You will have a question and the answer that student writes. The given QUESTION is VERIFIED and is ALWAYS CORRECT. STICK TO THE GIVEN QUESTION \
                  YOUR TASK: Guide and help your STUDENT to find correct answer and explaination to math questions. Never forget your TASK. \
                  You as a TEACHER will check if the explaination of the answer provided by the STUDENT is correct or not and provide him feedback. \
                  If the explaination is incorrect you will tell which part of the answer did the STUDENT went wrong, whether it is calculation or the approach the student used to get the answer \
                  If the explaination is correct you will praise the STUDENT and tell him to write the answer again. \
                  Also instruct the student every time to follow the given template \
                  TEMPLATE: \
                  Explanation Body Paragraph\
                  **Final Answer: <SINGLE NUMERICAL ANSWER>** \
                  Make sure the last line of any response is ALWAYS - **Final Answer: <SINGLE NUMERICAL ANSWER>** "}



In [ ]:
model_name = 'mixtral-8x7b-32768'
def multi_agent_chat(query):
    student_chat = []
    teacher_chat = []
    student_chat.append(student_system)
    teacher_chat.append(teacher_system)
    for i in range(2):
        # print('___________________________________________')
        student_chat.append({"role": "user", "content": query})
        teacher_chat.append({"role": "user", "content": query})
        student_response = client.chat.completions.create(model=model_name, messages=student_chat, temperature=0)
        # print(student_response.choices[0].message.content)
        teacher_chat.append({"role": "user", "content": student_response.choices[0].message.content})
        teacher_response = client.chat.completions.create(model=model_name, messages=teacher_chat, temperature=0, max_tokens=2048)
        # print(teacher_response.choices[0].message.content)
        student_chat.append({"role": "user", "content": teacher_response.choices[0].message.content})
        student_response = client.chat.completions.create(model=model_name, messages=student_chat, temperature=0, max_tokens=2048)
        # print('___________________________________________')
    # print("FINAL RESPONSE")
    # print(student_response.choices[0].message.content)
    return student_response.choices[0].message.content

In [ ]:
import re
import time
global_counter = 0

def ans_extractor(output):
    global global_counter
    matcha = re.search(r'Answer:\s*([0-9]+)', output)
    if matcha:
        final_answer = matcha.group(1)
        return final_answer
    else:
        global_counter += 1
        return 0

In [106]:
pt = []
gt = []
counter=0
for i in ds['test']:
    time.sleep(2)
    counter+=1
    print(counter, global_counter)
    query = i['question_concat']
    explaination = multi_agent_chat(query)
    answer = ans_extractor(explaination)
    if answer!=None:
        pt.append(answer)
        gt.append(i['Answer'])

1 0
2 0
3 0
4 0
5 1
6 1
7 1
8 1
9 2
10 2
11 2
12 2
13 2
14 2
15 3
16 3
17 3
18 3
19 3
20 3
21 3
22 3
23 3
24 3
25 4
26 5
27 5
28 5
29 5
30 5
31 6
32 6
33 6
34 6
35 7
36 7
37 7
38 7
39 7
40 7
41 7
42 7
43 8
44 8
45 8
46 8
47 8
48 9
49 10
50 10
51 10
52 10
53 10
54 11
55 11
56 11
57 11
58 11
59 11
60 11
61 12
62 13
63 13
64 13
65 13
66 13
67 13
68 13
69 13
70 13
71 13
72 13
73 13
74 14
75 14
76 14
77 14
78 14
79 14
80 14
81 14
82 14
83 14
84 14
85 14
86 14
87 14
88 14
89 14
90 14
91 15
92 15
93 15
94 15
95 16
96 16
97 16
98 17
99 17
100 17
101 17
102 17
103 18
104 18
105 18
106 18
107 18
108 18
109 19
110 20
111 20
112 20
113 20
114 20
115 21
116 21
117 22
118 22
119 22
120 22
121 22
122 22
123 22
124 22
125 22
126 23
127 24
128 24
129 24
130 24
131 24
132 24


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01j89nvkdne3kvm03vm5j8m2j7` on : Limit 500000, Used 500014, Requested 483. Please try again in 1m25.8844s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [107]:
from sklearn.metrics import accuracy_score
print(global_counter)
accuracy = accuracy_score(gt, pt)
print(f'Accuracy: {accuracy:.2f}')

24
Accuracy: 0.66
